In [2]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm, tqdm_notebook,tqdm_pandas
import itertools
import re
import warnings
warnings.filterwarnings(action='ignore')

### 분할이 된 파일들만 
분할이 된 파일들만 통합하고 각 폴더밖으로 꺼내준다 

In [4]:
# path = './output/token/'
path = './output/token/'
file_list = os.listdir(path)
folder_list = []
for file in file_list:
    if os.path.splitext(file)[1] != '.csv':
        folder_list.append(file)

for folder in tqdm_notebook(folder_list):
    file_list = os.listdir(f'{path}/{folder}/')
    all_df = pd.DataFrame()
    for file in file_list:
        read_file = pd.read_csv(f'{path}/{folder}/{file}')
        all_df = pd.concat([all_df,read_file])
        size = all_df.shape[0]
    all_df.to_csv(f'./output/token/{folder}_0~{size}.csv',index=False)


### 파일결합
* 검색어를 다수로 사용했기에 각 파일을 마을별로 묶어준다

In [29]:
path = './output/token/'
file_list = os.listdir(path)
csv_list = []
for file in file_list:
    if os.path.splitext(file)[1] == '.csv':
        csv_list.append(file)

In [30]:
merge_file = dict()
merge_file['물치'] = ['물치리','물치','물치항']
merge_file['전진2리'] = ['전진리','설악해수욕장','후진항']
merge_file['낙산'] = ['낙산','낙산항','낙산해수욕장']
merge_file['오산'] = ['오산','오산해수욕장','오산리']
merge_file['수산'] = ['수산리','수산항'] #'수산'때문에 망한듯
merge_file['동호'] = ['동호','동호리','동호해변','동호해수욕장']
merge_file['하광정'] = ['하광정리','하조대','하조대해수욕장']
merge_file['기사문'] = ['기사문리','기사문','기사문항','기사문해변']
merge_file['동산'] = ['동산','동산리','동산항','동산해수욕장']
merge_file['인구1리'] = ['인구리','죽도'] # 검색어 미쳐날뜀 '인구'때문인듯
merge_file['광진'] = ['광진리','광진','광진해변','휴휴암']
merge_file['남애리'] = ['남애리','남애','남애항']

In [31]:
#결합해야하는 파일 솎아내기
files = dict()
for file in csv_list:
    try:
        keyword = file.split('+')[0].replace(' ', '')
        for key in merge_file.keys():
            if keyword in merge_file[key]:
                if files.get(key) == None:
                    files[key] = [file]
                else:
                    files[key].append(file)
    except:
        pass
    
#파일결합
for key in files.keys():
    merged_df = pd.DataFrame()
    for file in files[key]:
        file_df = pd.read_csv(path + file)
        merged_df = pd.concat([merged_df,file_df])
    merged_df = merged_df.drop_duplicates('full_text')
    
    #필터링
    merged_df = merged_df.fillna('')
    filtered = merged_df.full_text.apply(lambda x : any(ele in x for ele in filter_word))
    clean_file = merged_df[~filtered]
    filtered = clean_file.title.apply(lambda x : any(ele in x for ele in filter_word))
    clean_file = clean_file[~filtered]
    
    total_len = merged_df.shape[0]
    clean_file.to_csv(f'./output/token_통합/{key}_{total_len}.csv',index = False)

### 파일 종합 결과

In [27]:
path = './output/token_통합/'
file_list = os.listdir(path)

total_df = pd.DataFrame()
for file in file_list:
    sumup = dict()
    keyword = file.split('_')[0]
    df = pd.read_csv(path +file)
    sumup['total_size'] = df.shape[0]
    for year in range(2010,2020):
        try :
            sumup[year] = df[df.year == year].shape[0]
        except :
            sumup[year] = np.nan
    one_df = pd.DataFrame(sumup,index =[keyword])
    total_df = pd.concat([total_df,one_df])

In [28]:
total_df

,total_size,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
강릉,53832,2938,3791,4520,4979,5810,6441,6270,6270,6361,6452
강원도,61330,5321,5705,5847,5999,6413,6925,6645,6682,6102,5691
광진,1282,42,81,73,97,124,142,176,159,161,227
기사문,2595,33,66,82,136,257,364,362,386,404,505
낙산,20460,670,883,922,1100,1573,2164,2446,2902,3082,4718
남애리,5377,165,173,230,280,467,669,734,976,859,824
동산,8952,314,394,495,691,887,1082,974,1104,1371,1640
동호,1911,15,19,21,42,87,219,234,375,440,459
물치,4564,160,153,221,247,321,438,519,747,907,851
속초,57240,2653,3709,4536,5250,6490,7076,7005,6996,6757,6768


### 각각탐색

In [33]:
path = './output/token_통합/'
file_list = os.listdir(path)

In [34]:
file_list

['강릉_53832.csv',
 '강원도_61330.csv',
 '광진_1282.csv',
 '기사문_2595.csv',
 '낙산_20460.csv',
 '남애리_5377.csv',
 '동산_8952.csv',
 '동호_1911.csv',
 '동호_3826.csv',
 '물치_4564.csv',
 '속초_57240.csv',
 '수산_2638.csv',
 '양양_47915.csv',
 '오산_6409.csv',
 '인구1리_8698.csv',
 '전진2리_2566.csv',
 '하광정_17375.csv']

In [53]:
sample = pd.read_csv(path +'수산_2638.csv')

In [54]:
sample.title.value_counts().head(20)

강원도 양양 수산항...         58
쏠비치 양양                49
양양 수산어촌체험마...         39
강원도 양양 수산항 엑스포...     36
1004bd님의블로그           32
아침나라/안영국              28
누룽지게스트하우스...          26
양양 수산어촌체험마을           23
양양군청 공식블로그            21
동해안 바다 속초 양양 낙산...    19
.                     16
양양 배낚시&펜션             14
임진강나룻터                13
강원타임즈 블로그             13
피싱곰과 함께하는 ...         13
지마와 신휴의 낚시...         13
쏠비치맛집(낙산 바...         12
Claire & Sky          11
피사체에 담은 muse...       10
양양 속초 낙산 동호...         9
Name: title, dtype: int64